In [1]:
import json
import spacy
import numpy as np
import shutil
import os

In [2]:
BASE_DIR = './newsdata'

In [3]:
# Get captions 
f = open(BASE_DIR+'/news_dataset.json')
caption_data_raw = json.load(f)
f.close()

In [88]:
def get_captions(data,indices):
    # Get captions at specified indices and do NER on the captions
    caption_data_cleaned =[["image","caption"]]
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("merge_entities")
    if not os.path.exists(BASE_DIR+'/Images/'): 
        os.makedirs(BASE_DIR+'/Images/')
    else:
        for file in os.listdir(BASE_DIR+'/Images/'):
            os.remove(BASE_DIR+'/Images/'+file)
    for i in indices:
        entry = data[i]
        caption = entry['sentences_full'][0]['raw']
        doc = nlp(caption)
        caption_ner = " ".join([t.text if not t.ent_type_ else '<'+t.ent_type_+'>' for t in doc])
        image_name = entry['filename']
        image_path = BASE_DIR+'/all_images/'+image_name
        if os.path.isfile(image_path):
            shutil.copy(image_path, BASE_DIR+'/Images/'+image_name)
        else:
            continue
        entry_cleaned = [image_name,caption_ner]
        caption_data_cleaned.append(entry_cleaned)
    return np.array(caption_data_cleaned)

In [119]:
np.random.seed(42)
indices = np.random.choice(range(len(caption_data_raw)), size=8000, replace=False)
indices

array([461404, 403235, 236893, ...,  64352,  47588,  23483])

In [120]:
dataset = get_captions(caption_data_raw,indices)
np.savetxt(BASE_DIR+"/news_captions_ner.txt", dataset,delimiter=',',fmt="%s")
print(dataset)

[['image' 'caption']
 ['59838df695d0e0246f1fdff4_3.jpg'
  '<PERSON> Ring acrylic on papiermch from <DATE> On the wall Jayson Mussons <ORG> gouache on coffeetinted paper from <DATE> at <PERSON>']
 ['52dda2c438f0d808a7480e23_1.jpg'
  'The Knicks <PERSON> battling <PERSON> said the Nets continually found mismatches']
 ...
 ['52f0522a38f0d80f67aeba9a_3.jpg'
  '<GPE> taking her gear to the teams locker room at the alley']
 ['5a60a82d95d0e0110c1a5379_0.jpg'
  '<PERSON> the former <NORP> governor of <GPE> sorted through tickets to <PERSON> shows that he had saved at his home in <DATE>']
 ['541bd60238f0d8296cb1029c_0.jpg'
  'An invitation to an event in <DATE> that is being held by <ORG> network of donors']]
